Import Necessary Modules

In [1]:
import h2o
import swat
import pandas as pd
from _config import login,container_login
from swat.render import render_html
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from IPython.display import HTML,IFrame
%matplotlib inline
from plotnine import *

Start the h2o server if not already available

In [2]:
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_161"; OpenJDK Runtime Environment (build 1.8.0_161-b14); OpenJDK 64-Bit Server VM (build 25.161-b14, mixed mode)
  Starting server from /opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmppcuf6uh6
  JVM stdout: /tmp/tmppcuf6uh6/h2o_centos_started_from_python.out
  JVM stderr: /tmp/tmppcuf6uh6/h2o_centos_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,1 month and 18 days
H2O cluster name:,H2O_from_python_centos_u4ejqf
H2O cluster total nodes:,1
H2O cluster free memory:,6.948 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Import the sample data

In [3]:
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#h2o df
train.head()

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
1,0.869293,-0.635082,0.22569,0.32747,-0.689993,0.754202,-0.248573,-1.09206,0,1.37499,-0.653674,0.930349,1.10744,1.1389,-1.5782,-1.04699,0,0.65793,-0.0104546,-0.0457672,3.10196,1.35376,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.907542,0.329147,0.359412,1.49797,-0.31301,1.09553,-0.557525,-1.58823,2.17308,0.812581,-0.213642,1.27101,2.21487,0.499994,-1.26143,0.732156,0,0.398701,-1.13893,-0.00081911,0,0.30222,0.833048,0.9857,0.978098,0.779732,0.992356,0.798343
1,0.798835,1.47064,-1.63597,0.453773,0.425629,1.10487,1.28232,1.38166,0,0.851737,1.54066,-0.81969,2.21487,0.99349,0.35608,-0.208778,2.54822,1.25695,1.12885,0.900461,0,0.909753,1.10833,0.985692,0.951331,0.803252,0.865924,0.780118
0,1.34438,-0.876626,0.935913,1.99205,0.882454,1.78607,-1.64678,-0.942383,0,2.42326,-0.676016,0.736159,2.21487,1.29872,-1.43074,-0.364658,0,0.745313,-0.678379,-1.36036,0,0.946652,1.0287,0.998656,0.728281,0.8692,1.02674,0.957904
1,1.10501,0.321356,1.5224,0.882808,-1.20535,0.681466,-1.07046,-0.921871,0,0.800872,1.02097,0.971407,2.21487,0.596761,-0.350273,0.631194,0,0.479999,-0.373566,0.113041,0,0.755856,1.36106,0.98661,0.838085,1.1333,0.872245,0.808487
0,1.59584,-0.607811,0.00707492,1.81845,-0.111906,0.84755,-0.566437,1.58124,2.17308,0.755421,0.64311,1.42637,0,0.921661,-1.19043,-1.61559,0,0.651114,-0.654227,-1.27434,3.10196,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
1,0.409391,-1.88468,-1.02729,1.67245,-1.6046,1.33801,0.0554274,0.0134659,2.17308,0.509783,-1.03834,0.707862,0,0.746918,-0.358465,-1.64665,0,0.367058,0.0694965,1.37713,3.10196,0.869418,1.22208,1.00063,0.545045,0.698653,0.977314,0.828786
1,0.933895,0.62913,0.527535,0.238033,-0.966569,0.547811,-0.0594392,-1.70687,2.17308,0.941003,-2.65373,-0.15722,0,1.03037,-0.175505,0.523021,2.54822,1.37355,1.29125,-1.46745,0,0.901837,1.08367,0.979696,0.7833,0.849195,0.894356,0.774879
1,1.40514,0.536603,0.689554,1.17957,-0.110061,3.2024,-1.52696,-1.57603,0,2.93154,0.567342,-0.130033,2.21487,1.78712,0.899499,0.585151,2.54822,0.401865,-0.151202,1.16349,0,1.66707,4.03927,1.17583,1.04535,1.54297,3.53483,2.74075
1,1.17657,0.104161,1.397,0.479721,0.265513,1.13556,1.53483,-0.253291,0,1.02725,0.534316,1.18002,0,2.40566,0.0875568,-0.976534,2.54822,1.25038,0.268541,0.530334,0,0.833175,0.773968,0.98575,1.1037,0.84914,0.937104,0.812364


Export Data as dataframe

In [5]:
pd_training_df = train.as_data_frame()
pd_test_df = test.as_data_frame()
# pd_training_df["response"]=pd_training_df["response"].astype("object")

In [6]:
pd_training_df.describe()

,response,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
count,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,...,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,0.53,1.00,-0.02,-0.00,0.99,-0.01,0.99,-0.00,0.00,1.00,...,0.00,-0.01,1.00,1.03,1.02,1.05,1.01,0.97,1.03,0.96
std,0.50,0.57,1.00,1.01,0.60,1.00,0.48,1.00,1.02,1.03,...,1.01,1.00,1.40,0.64,0.37,0.17,0.40,0.52,0.37,0.31
min,0.00,0.28,-2.43,-1.74,0.01,-1.74,0.16,-2.94,-1.74,0.00,...,-2.50,-1.74,0.00,0.17,0.34,0.46,0.38,0.08,0.39,0.44
25%,0.00,0.60,-0.74,-0.87,0.57,-0.89,0.68,-0.68,-0.89,0.00,...,-0.71,-0.89,0.00,0.79,0.85,0.99,0.77,0.68,0.82,0.77
50%,1.00,0.86,-0.03,0.00,0.89,-0.02,0.90,-0.01,0.02,1.09,...,-0.00,-0.01,0.00,0.89,0.95,0.99,0.92,0.87,0.95,0.87
75%,1.00,1.25,0.71,0.88,1.29,0.86,1.17,0.68,0.88,2.17,...,0.72,0.85,3.10,1.02,1.08,1.02,1.15,1.12,1.13,1.06
max,1.00,6.70,2.43,1.74,5.82,1.74,7.06,2.97,1.74,2.17,...,2.50,1.74,3.10,13.10,7.39,3.68,6.58,8.26,4.75,4.32


Connect to CAS and Check CAS Cluster Status

In [7]:
host = container_login()[2]
user = container_login()[0]
pswd = container_login()[1]
conn = swat.CAS(host,5571,user,pswd)

conn.about()

NOTE: Grid node action status report: 1 nodes, 8 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'Version': '3.04',
  'VersionLong': 'V.03.04M0P07112018',
  'Copyright': 'Copyright © 2014-2018 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2019-05-19T16:58:28Z',
  'System': {'Hostname': 'sas-programming',
   'OS Name': 'Linux',
   'OS Family': 'LIN X64',
   'OS Release': '3.10.0-693.5.2.el7.x86_64',
   'OS Version': '#1 SMP Fri Oct 20 20:32:50 UTC 2017',
   'Model Number': 'x86_64',
   'Linux Distribution': 'CentOS Linux release 7.5.1804 (Core)'},
  'license': {'site': 'poac',
   'siteNum': 70180938,
   'expires': '01Apr2020:00:00:00',
   'gracePeriod': 45,
   'warningPeriod': 45,
   'maxCPUs': 9999}}

[server]

 Server Status
 
    nodes  actions
 0      1        8

[nodestatus]

 Node Status
 
               name        role  uptime  running  stalled
 0  sas-programming  controller    0.26        0        0

+ Elapsed: 0.00128s, sys: 0.00125s, mem: 0.295mb

Perform AutoML on H2o

In [7]:
# Identify predictors and response
x = train.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190519_151644,0.790013,0.551199,0.326536,0.432095,0.186706
StackedEnsemble_BestOfFamily_AutoML_20190519_151644,0.78792,0.553113,0.338466,0.432956,0.187451
XGBoost_grid_1_AutoML_20190519_151644_model_3,0.784333,0.558431,0.334059,0.435102,0.189314
XGBoost_grid_1_AutoML_20190519_151644_model_4,0.783708,0.557602,0.318335,0.435124,0.189333
XGBoost_1_AutoML_20190519_151644,0.782733,0.556467,0.330409,0.435054,0.189272
XGBoost_3_AutoML_20190519_151644,0.781994,0.564004,0.315146,0.437432,0.191347
XGBoost_2_AutoML_20190519_151644,0.78123,0.559717,0.318748,0.436135,0.190213
GBM_5_AutoML_20190519_151644,0.780837,0.559903,0.340848,0.436191,0.190263
GBM_2_AutoML_20190519_151644,0.780036,0.559806,0.339926,0.436415,0.190458
GBM_1_AutoML_20190519_151644,0.779827,0.560857,0.335096,0.436616,0.190633


Upload DataFrame to CAS

In [8]:
cas_table = conn.upload(pd_training_df,casout = dict(name="ml_frame", caslib="Public", replace=True))

NOTE: Cloud Analytic Services made the uploaded file available as table ML_FRAME in caslib Public.
NOTE: The table ML_FRAME has been created in caslib Public from binary data uploaded to Cloud Analytic Services.


In [9]:
score_table = conn.upload(pd_test_df,casout = dict(name="score_ml_frame", caslib="Public", replace=True))['casTable']

NOTE: Cloud Analytic Services made the uploaded file available as table SCORE_ML_FRAME in caslib Public.
NOTE: The table SCORE_ML_FRAME has been created in caslib Public from binary data uploaded to Cloud Analytic Services.


In [10]:
cas_table

[caslib]

 'Public'

[tableName]

 'ML_FRAME'

[casTable]

 CASTable('ML_FRAME', caslib='Public')

+ Elapsed: 0.109s, user: 0.15s, sys: 0.0435s, mem: 15.9mb

In [11]:
cas_table = cas_table['casTable']
cas_table.head()

,response,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
0,1.00,0.87,-0.64,0.23,0.33,-0.69,0.75,-0.25,-1.09,0.00,...,-0.01,-0.05,3.10,1.35,0.98,0.98,0.92,0.72,0.99,0.88
1,1.00,0.91,0.33,0.36,1.50,-0.31,1.10,-0.56,-1.59,2.17,...,-1.14,-0.00,0.00,0.30,0.83,0.99,0.98,0.78,0.99,0.80
2,1.00,0.80,1.47,-1.64,0.45,0.43,1.10,1.28,1.38,0.00,...,1.13,0.90,0.00,0.91,1.11,0.99,0.95,0.80,0.87,0.78
3,0.00,1.34,-0.88,0.94,1.99,0.88,1.79,-1.65,-0.94,0.00,...,-0.68,-1.36,0.00,0.95,1.03,1.00,0.73,0.87,1.03,0.96
4,1.00,1.11,0.32,1.52,0.88,-1.21,0.68,-1.07,-0.92,0.00,...,-0.37,0.11,0.00,0.76,1.36,0.99,0.84,1.13,0.87,0.81


Prep Variables, Run Autotune

In [12]:
# all input variables are numeric
input_vars = {i for i in cas_table.columns if 'x' in i}
target_var = "response"

# explore_action_set = conn.actionsetinfo(all=True)["setinfo"]
conn.loadactionset("decisionTree")
conn.loadactionset("autotune")

NOTE: Added action set 'decisionTree'.
NOTE: Added action set 'autotune'.


[actionset]

 'autotune'

+ Elapsed: 0.002s, user: 0.000962s, sys: 0.000994s, mem: 0.213mb

Run Autotune Actions

In [13]:
result = conn.autotune.tuneGradientBoostTree(
           trainOptions = {
              "table"   : cas_table,
              "inputs"  : input_vars,
              "target"  : target_var,
              "casout"  : {"name":"gradboost_ml_model"},
               "maxLevel":7,
               "quantileBin":True,
               "nbins":256,
               "missing":"USEINSEARCH",
               "learningRate":0.1,
               "ridge":1,
               "subSampleRate":1,
               "nTree":50,
               "greedy":True
           },
           tunerOptions= dict(seed=12345)
      )

NOTE: Autotune is started for 'Gradient Boosting Tree' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=50.
NOTE: Autotune option MAXTIME=36000 (sec.).
NOTE: Autotune option SEED=12345.
NOTE: Autotune objective is 'Mean Square Error'.
NOTE: Autotune number of parallel evaluations is set to 4, each using 0 worker nodes.
NOTE: Automatic early stopping is activated with STAGNATION=4;  set EARLYSTOP=false to deactivate.
         Iteration       Evals     Best Objective        Time
                 0           1             0.1965        9.49
                 1          10             0.1965       33.30
                 2          19             0.1956       93.19
                 3          28             0.1918      163.18
                 4          37             0.1917      260.20
                 5          46             0.1917      367.05
NOTE: Data was partitioned during tuning, to tune based on validation score; the final model is trained and scored o

In [14]:
result

,Descr,Value
0,Number of Trees,84.00
1,Distribution,1.00
2,Learning Rate,0.05
3,Subsampling Rate,0.57
4,Number of Selected Variables (M),19.00
5,Number of Bins,256.00
6,Number of Variables,28.00
7,Max Number of Tree Nodes,125.00
8,Min Number of Tree Nodes,67.00
9,Max Number of Branches,2.00


*Other Similar Actions*

Today you can run auto-tune for **RandomForest or NeuralNet or DecisionTrees**
```
# This is an example for autotuning Random forest
result_forest = conn.autotune.tuneForest(
           trainOptions = {
              "table"   : cas_table,
              "inputs"  : input_vars,
              "target"  : target_var,
              "casout"  : {"name":"forest_ml_model"}
           },
           tunerOptions= dict(seed=54321)
      )


# This is an example for autotuning Neural Nets
result_nnet = conn.autotune.tuneNeuralNet(
           trainOptions = {
              "table"   : cas_table,
              "inputs"  : input_vars,
              "target"  : target_var,
              "casout"  : {"name":"nn_ml_model"}
           },
           tunerOptions={"seed":45321}
      )
      

# This is an example for autotuning Decision Trees

result_dree = conn.autotune.tuneDecisionTree(
           trainOptions = {
              "table"   : cas_table,
              "inputs"  : input_vars,
              "target"  : target_var,
              "casout"  : {"name":"dt_ml_model"}
           },
           tunerOptions={"seed":46321}
      )

```




Now lets run a Scikit-Learn Model just to show we can run this on the same server

In [17]:
X_train = pd_training_df.iloc[:,1:29].values
y_train = pd_training_df.iloc[:,0].values
X_test = pd_test_df.iloc[:,1:29].values
y_test = pd_test_df.iloc[:,0].values

In [18]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Get MSE  - Brier Score comparison
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred))

0.3334


Close Connections to Instances that are running!

In [20]:
conn.close()
h2o.cluster().shutdown()

H2O session _sid_a923 closed.


**19WK21 Release (May 21st 2019)** has a full Auto-ML including the pre-processing and the post processing steps
![title](dataSciencePilot.png)